In [ ]:
!pip install langchain langgraph openai

In [ ]:
from typing import Annotated, List, Tuple

from typing_extensions import TypedDict
import operator
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_core.prompts import PromptTemplate

planner_template = PromptTemplate.from_template(
    """You are a planning agent tasked with converting the user's objective into a sequence of detailed, logical steps.
        You specialize in the creation of conceptual maps for complex topics. Whenever a task is complex and requires multiple steps, subdivide it into smaller tasks.
        For example when the task involves the summarization of a large book, you may want to break it down into multiple steps.

        Example1: ["Describe relevant characters in the story", "list all chapters", "summarize chapters 1 - 10", "summarize chapters 11 - 20", "summarize chapters 21 - 37"]

        task: 
        {task_instruction}

        summary:
        {summary}

        Return a JSON array describing the ordered steps needed to complete the task. Each step should include:
        - step_id
        - description
        - expected_input
        - expected_output

        output: [{{json_array}}]
"""
)



replanner_template = PromptTemplate.from_template(
    """Your job is to check that the previously formulated plan is enought to properly solve the given task. Please make sure it is consistent and all the steps required by the user are covered by the current plan. Be meticoulous.
        
        plan:
        {plan}
        
        task: 
        {task_instruction}

        summary:
        {summary}

        Return a JSON array describing the ordered steps needed to complete the task. Each step should include:
        - step_id
        - description
        - expected_input
        - expected_output

        output: [{{json_array}}]
"""
)
summarizer_template = PromptTemplate.from_template(
    """You are an assistant tasked with summarizing input text for downstream use in a concept map generator.
        Summarize the key topics and structure in no more than 300 words, highlighting relationships or dependencies between concepts.
        Output the result in plain text. The summary should be long enough to answer precisely the question of the user.
            
        Always make sure that all available information relevant to the task is included. 
        When there are important entities, describe them and their role in the story.
        When there are chapters, summarize them in blocks (I-VII, VIII-XII, ... XXX - XXXVII and so on).

        user request:
        {task_instruction}

        context:
        {context}
""")


executor_template = PromptTemplate.from_template(
   """You are an executor agent. Your goal is to execute each step of the provided plan using the given summary of the input context.

    previous response:
    {response}

    plan step:
    {plan_step}

    summary:
    {summary}

    context:
    {context}

    return in output just the final user-facing response that comes from the execution of the plan. Be sure the expected output is consistent with the requirements of the plan step.
    The intermediate response should be formatted in Markdown format.
    
    response: {{response}}

""")

parser_template = PromptTemplate.from_template(
"""
    You are an expert parser who is going to produce a JSON structure encoding the final response provided by the reasoning part of the LLM into a conceptual map.

    instruction: {task_instruction}
    
    response: {response}
    
    output in JSON format as output of nested topics. If the instructions contain format requirements, try to implement them, but be sure not to modify the tree structure of the map: 
    - main_topic_keyword
        - subtopic_1_keyword
            - leaf_topic_keyword: "text description"
        - subtopic_2_keyword
            - leaf_topic_keyword: "text description"
"""
)

In [ ]:
from langgraph.managed import RemainingSteps

class PlanExecute(TypedDict):
    context: str
    task_instruction: str
    summary: str
    plan: str
    plan_step: str
    messages: Annotated[List[str], add_messages]
    remaining_steps: RemainingSteps
    response: str

planner = create_react_agent(
    model = "openai:gpt-4.1",
    tools=[],
    name="map_planner",
    state_schema=PlanExecute,
    prompt=planner_template
)

replanner = create_react_agent(
    model = "openai:gpt-4.1",
    tools=[],
    name="map_replanner",
    state_schema=PlanExecute,
    prompt=planner_template
)

executor = create_react_agent(
    model = "openai:gpt-4.1",
    tools=[],
    name="map_writer",
    state_schema=PlanExecute,
    prompt=executor_template
)

summarizer = create_react_agent(
    model = "openai:gpt-4.1",
    tools=[],
    name="text_summarizer",
    state_schema=PlanExecute,
    prompt=summarizer_template
)

parser = create_react_agent(
    model = "openai:gpt-4.1",
    tools=[],
    name="map_producer",
    state_schema=PlanExecute,
    prompt=parser_template
)

In [ ]:
import json
import re

def planning_node(state: PlanExecute):
    return {"plan": planner.invoke(state)['messages'][-1].content}

def replanning_node(state: PlanExecute):
    plan = state['plan']
    for i in range(3):
        plan = replanner.invoke(state)['messages'][-1].content
        state['plan'] = plan
    return {"plan": replanner.invoke(state)['messages'][-1].content}

def summary_node(state: PlanExecute):
    return {"summary": summarizer.invoke(state)['messages'][-1].content}

def executor_node(state: PlanExecute):

    plan = state['plan']
    plan = re.search(r'\[.*\]', plan, re.DOTALL).group(0)

    plan_dict = json.loads(plan)
    
    resp = ""

    for plan_step in plan_dict:
        state['plan_step'] = plan_step
        state['response'] = resp
        resp = executor.invoke(state)['messages'][-1].content

    return {"response": resp}

def parser_node(state: PlanExecute):
    return {"response": parser.invoke(state)['messages'][-1].content}

In [ ]:
from IPython.display import Image, display

graph_builder = StateGraph(PlanExecute)

# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("summarizer", summary_node)
graph_builder.add_node("planner", planning_node)
graph_builder.add_node("replanner", replanning_node)
graph_builder.add_node("executor", executor_node)
graph_builder.add_node("parser", parser_node)

graph_builder.add_edge(START, "summarizer")
graph_builder.add_edge("summarizer", "planner")
graph_builder.add_edge("planner", "replanner")
graph_builder.add_edge("replanner", "executor")
graph_builder.add_edge("executor", 'parser')
graph_builder.add_edge("parser", END)

graph = graph_builder.compile()

# display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
file = open('./i_promessi_sposi.txt', 'r')
context = file.read()
file.close()
len(context)

In [ ]:
instruction = """Genera una mappa in inglese, tono divulgativo. Ogni sezione principale deve contenere il titolo e numero del capitolo nel fomato "Capitolo ZZZZ: titolo" e ogni sottosessione deve essere dedicata alla storia di un personaggio in quel capitolo più una sezione bonus con il riassunto breve del capitolo intitolata "summary". CREA UN NODO PER OGNI SINGOLO CAPITOLO"""

In [ ]:
final_state = None

for event in graph.stream({"context": context, "task_instruction": instruction}):
    for value in event.values():
        print(value)
        final_state = value

In [ ]:
response = final_state['response']
response = re.search(r"{.*}", response, re.DOTALL).group(0)
concept_map = json.loads(response)

print(concept_map)

In [ ]:
# Generalized function to render any JSON tree with unlimited layers and fun styling
def render_json_tree_html(data, level=0):
    html = ""
    indent = "  " * level
    if isinstance(data, dict):
        for key, value in data.items():
            display_key = str(key).replace("_", " ").capitalize()
            emoji = "🧠" if level == 0 else "📂"
            html += f"{indent}<details open><summary>{emoji} <strong>{display_key}</strong> 🌟</summary>\n"
            html += render_json_tree_html(value, level + 1)
            html += f"{indent}</details>\n"
    elif isinstance(data, list):
        for i, item in enumerate(data):
            html += f"{indent}<details open><summary>📚 <strong>Item {i+1}</strong></summary>\n"
            html += render_json_tree_html(item, level + 1)
            html += f"{indent}</details>\n"
    else:
        html += f"{indent}<p class='text-blurb'>🔸 {str(data)}</p>\n"
    return html

# Wrap it in a magical HTML document
generic_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>🧩 Conceptual Map </title>
<style>
  @import url('https://fonts.googleapis.com/css2?family=Indie+Flower&family=Fredoka+One&display=swap');

  body {{
    font-family: 'Indie Flower', cursive;
    background: linear-gradient(to right, #fbc2eb, #a6c1ee);
    color: #2d2d2d;
    padding: 40px;
  }}
  h1 {{
    font-family: 'Fredoka One', cursive;
    font-size: 3em;
    text-align: center;
    color: #ff006e;
    margin-bottom: 30px;
  }}
  details {{
    background: #ffffffc9;
    border-radius: 10px;
    border: 2px dashed #ff70a6;
    padding: 12px;
    margin-bottom: 10px;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.05);
  }}
  details:hover {{
    background-color: #fff0f6;
  }}
  summary {{
    font-size: 1.2em;
    color: #5c2a9d;
    cursor: pointer;
    font-weight: bold;
  }}
  .text-blurb {{
    margin: 8px 0 8px 20px;
    font-size: 1.05em;
    background: #fdfcdc;
    padding: 10px;
    border-left: 4px solid #ffd166;
    border-radius: 6px;
  }}
</style>
</head>
<body>
<h1>🧩 Conceptual Map </h1>
{render_json_tree_html(concept_map)}
<p style='text-align:center;'>🔍 This tree adapts to <strong>any</strong> JSON structure — deeply nested, irregular, or totally wild!</p>
</body>
</html>
"""

# Save to file
generic_html_path = "concept_map.html"
with open(generic_html_path, "w", encoding="utf-8") as f:
    f.write(generic_html)

generic_html_path